In [34]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

def train_stacking_model(X, y, event_types, event_weights, run_id):
    print(f"\n🟢 Training Stacking Model {run_id}...\n")

    X_train, X_test, y_train, y_test, event_train, event_test = train_test_split(
        X, y, event_types, test_size=0.2, # random split everytime
    )

    print("Fitting LGBM-RF")
    lgbm_rf_model.fit(X_train, y_train)
    print("Fitting LGBM")
    lgbm_model.fit(X_train, y_train)
    print("Fitting XGBoost")
    xgb_model.fit(X_train, y_train)
    print("Fitting MLP")
    mlp_model.fit(X_train, y_train)
    print("Models refitted on the new dataset")

    meta_xgb = XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        colsample_bytree=0.8,
        subsample=0.8,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    )

    base_learners = {
        "LGBM_RF": lgbm_rf_model,
        "LGBM": lgbm_model,
        "XGBoost": xgb_model,
        "MLP": mlp_model
    }

    stacking_model = StackingClassifier(
        estimators=[(name, model) for name, model in base_learners.items()],
        final_estimator=meta_xgb,
        cv="prefit",
        n_jobs=-1, 
        passthrough=False 
    )

    stacking_model.fit(X_train, y_train)
    y_pred = stacking_model.predict(X_test)
    
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
    
    # calculate best signal significance
    y_scores = stacking_model.predict_proba(X_test)[:, 1]

    best_threshold = 0
    best_significance = 0

    for threshold in np.linspace(0.05, 0.95, 50):  
        y_pred = (y_scores >= threshold).astype(int)  

        signal = sum(
            event_weights[event_type] * np.sum((y_pred[event_test == event_type] == 1) & (y_test[event_test == event_type] == 1))
            for event_type in event_weights.keys()
        )
        background = sum(
            event_weights[event_type] * np.sum((y_pred[event_test == event_type] == 1) & (y_test[event_test == event_type] == 0))
            for event_type in event_weights.keys()
        )

        if signal + background > 0:
            signal_significance = signal / np.sqrt(signal + background)
            
            if signal_significance > best_significance:
                best_significance = signal_significance
                best_threshold = threshold

    print(f"Best Threshold: {best_threshold:.3f}")
    print(f"Best Weighted Signal Significance): {best_significance:.4f}")

    # compute weighted background for chosen threshold
    y_pred = (y_scores >= best_threshold).astype(int)  

    surviving_background = {event: 0 for event in event_weights.keys()}
    total_weighted_FP = 0  

    for event_type in event_weights.keys():
        weight = event_weights[event_type] # get event weight
        event_mask = (event_test.values == event_type) 
        
        FP_count = np.sum((y_pred[event_mask] == 1) & (y_test.values[event_mask] == 0))  # false positives
        
        weighted_FP = FP_count * weight  
        surviving_background[event_type] = weighted_FP
        total_weighted_FP += weighted_FP  

    print(f"Total Weighted Surviving Background Events: {total_weighted_FP:.4f}")

    print("Weighted False Positives Per Background Type:")
    for event_type, fp_weight in surviving_background.items():
        print(f"  {event_type}: {fp_weight:.4f}")

    return best_significance, total_weighted_FP

# TRIALS
signal_significance_scores = []
weighted_background_scores = []

for run_id in range(20):
    significance, weighted_background = train_stacking_model(X, y, event_types, event_weights, run_id)
    signal_significance_scores.append(significance)
    weighted_background_scores.append(weighted_background)

mean_signal_significance = np.mean(signal_significance_scores)
variance_signal_significance = np.var(signal_significance_scores)

# Print final results
print("\nFinal Results Across 5 Models:")
print(f"Mean Signal Significance: {mean_signal_significance:.4f}")
print(f"Variance of Signal Significance: {variance_signal_significance:.4f}")
print(f"Mean Weighted Background: {mean_weighted_background:.4f}")
print(f"Variance of Weighted Background: {variance_weighted_background:.4f}")



🟢 Training Stacking Model 0...

Fitting LGBM-RF
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
[LightGBM] [Info] Number of positive: 184203, number of negative: 672981
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhe

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [02:56:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Training Complete
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Accuracy: 0.8842
✅ Recall: 0.6540
✅ F1 Score: 0.7093
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Best Threshold: 0.950
✅ Best Weighted Signal Significance (S/√(S+B)): 7.8194
✅ Total Weighted Surviving Backgroun

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [03:01:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Training Complete
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Accuracy: 0.8839
✅ Recall: 0.6554
✅ F1 Score: 0.7085
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Best Threshold: 0.932
✅ Best Weighted Signal Significance (S/√(S+B)): 8.1484
✅ Total Weighted Surviving Backgroun

...

✅ Training Complete
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Accuracy: 0.8847
✅ Recall: 0.6575
✅ F1 Score: 0.7110
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Best Threshold: 0.950
✅ Best Weighted Signal Significance (S/√(S+B)): 7.9495
✅ Total Weighted Surviving Backgroun

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [03:10:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Training Complete
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Accuracy: 0.8838
✅ Recall: 0.6551
✅ F1 Score: 0.7087
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Best Threshold: 0.950
✅ Best Weighted Signal Significance (S/√(S+B)): 8.4433
✅ Total Weighted Surviving Backgroun

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [03:14:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Training Complete
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Accuracy: 0.8849
✅ Recall: 0.6586
✅ F1 Score: 0.7106
[LightGBM] [Warning] feature_fraction is set=0.865132730509119, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.865132730509119
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] bagging_fraction is set=0.522571778164637, subsample=1.0 will be ignored. Current value: bagging_fraction=0.522571778164637
✅ Best Threshold: 0.932
✅ Best Weighted Signal Significance (S/√(S+B)): 8.0222
✅ Total Weighted Surviving Backgroun